# Interpolation

## Polynomial approximation

We'll write two series of functions to generate the interpolating polynomial as a vector in the space $\mathcal{P}_n$ of polynomials with degree at most $n$. The first will generate a basis of polynomials in the Lagrange form, the latter will do so in the Newton form. Both will then interpolate an arbitrary function $f \in C[a,b]$ at a finite number of points $x_i \in [a,b]$.

### Lagrange form

Consider $f \in C[a,b]$ (the set of continuous real-valued functions on the compact interval $[a,b]$), and let $x_0,\ldots,x_n$ be $n+1$ distinct points. Does there exist polynomial of least degree $p \in \RR[x]$ which *interpolates* $f$ at the given points? such that $f(x_i) = p(x_i)$ for all $i \in \{0,1,\ldots,n\}$? Is it unique?

The fundamental theorem of algebra guarantees uniqueness (the difference between any two least degree interpolating polynomials is the zero polynomial), and the Lagrange form of the interpolating polynomial demonstrates existence.

Having $n+1$ distinct points $x_i$ specified, the $n+1$ Lagrange polynomials $\ell_k$ are written as the product $$\ell_k(x) = \prod_{i\neq k} \frac{x-x_i}{x_k-x_i} \text{ for all } k \in \{0,1,\ldots,n\}.$$

That is:

In [ ]:
def lagrange_basis(vecx):
    n = len(vecx)
    # choose n-1 of n entries
    hinges = [np.concatenate([vecx[:i], vecx[i+1:]]) for i in range(n)]
    # take product of n-1 terms, format into lambda functions
    polys = [lambda t, i=i: np.prod((t-hinges[i])/(vecx[i]-hinges[i]))\
            for i in range(n)]
    # returns a list of n functions, the Lagrange basis polynomials
    return polys


It can be shown that the set of $n+1$ Lagrange polynomials spans $\mathcal{P}_n$. We note the Lagrange polynomials form a basis for $\mathcal{P}_n$ since $\dim(\mathcal{P}_n) = n+1$.

For example:

In [ ]:
vecx = np.array([0,1,2,3])

def lagrange_basis_plot(vecx):
    polys = lagrange_basis(vecx)
    start, stop = np.min(vecx), np.max(vecx)
    stiple = np.linspace(start,stop)
    # returns a list of plots of the basis polynomials, overlaid
    return [plt.plot(stiple,[polys[i](t) for t in stiple])\
            for i in range(len(vecx))]

lagrange_basis_plot(vecx)

Now, to show the existence of an interpolating polynomial, we'll specify a linear combination of basis vectors $\{\ell_k : k=0,1,\ldots,n\}$.

Let $f \in C[a,b]$, and specify $n+1$ distinct points $x_i$. The interpolating polynomial in Lagrange form is $p_n \in \mathcal{P}_n$ where $$p_n(x) = \sum_{k=0}^n f(x_k) \ell_k(x).$$

We'll scale our basis polynomials:

In [ ]:
def lagrange_coords(f,vecx):
    # returns the n coordinates of the interpolating polynomial
    # as a vector in $\mathcal{P}_n$ with the Lagrange basis
    return np.array([f(x) for x in vecx])

def interpol(coords,polys,t):
    # scales each vector by its coordinate
    scale = [lambda p, a=a: p * a for a in coords]
    # sums to form the desired linear combination
    # returns a (floating point) scalar
    return sum([scale[i](polys[i](t)) for i in range(len(vecx))])

Since the degree of any linear combination of Lagrange polynomials (generated by $n+1$ distinct points) is $n$, and evaluated at any of the $n+1$ generating points $x_i$, the Lagrange $k$th polynomial is equal to the Kroenecker delta $$\delta_{ik} = \begin{cases} 1 &\text{ if } i=k\\ 0 &\text{ else,} \end{cases}$$ we have the least degree interpolating polynomial that we sought the existence of, $p_n$.

Consider the function $f(x) = e^x$, and the points $\{0,1,2,3\}$. 

We can compute:

In [ ]:
import math

f = lambda x: math.exp(x)

# to specify the interpolating poly, p
polys = lagrange_basis(vecx)
coords = lagrange_coords(f,vecx)
p = interpol(coords, polys, t)

# here's a couple evaluations
# notice interpolation has small relative error
print("exp(1.5) is approx  ", p(1.5)) 
print("exp(1.5) is exactly ", math.exp(1.5))

# while exterpolation may have great relative error
print("exp(4.0) is approx  ", p(4.0))
print("exp(4.0) is exactly ", math.exp(4.0))

In [ ]:
# we sample 50 points in the interval
start,stop = np.min(vecx),np.max(vecx)
stiple = np.linspace(start,stop)

# evaluate them to make a plot for p
fstiple = [interpol(coords, polys, t) for t in stiple]
plt.plot(stiple,fstiple)

# and overlay the exact plot f
plt.plot(stiple,np.array([f(t) for t in stiple]))

### Newton form

The Newton form allows us to compute $p_n$ from $p_{n-1}$ recursively. That is, we define $p_n$ with the same $n-1$ basis vectors and $n-1$ coordinates as $p_{n-1}$, but add a basis vector that's $0$ at all points $x_i$ where $i < n$.

Namely, $$p_n(x) = p_{n-1}(x)+a_n\prod_{i < n} (x-x_i)$$ with $$a_n = \left(f(x_n) - p_{n-1}(x_n)\right)\left(\prod_{i < n} (x_n-x_i)\right)^{-1}.$$

Here's a constructive definition, with accompanying plots for the same generating points $\{0,1,2,3\}$.

In [ ]:
def newton_basis(vecx):
    n = len(vecx)
    # returns a list of n functions, the newton form basis polynomials
    return [lambda t, j=j: np.prod([t-vecx[i] for i in range(j)])\
            for j in range(n)]

def newton_basis_plot(vecx):
    polys = newton_basis(vecx)
    start, stop = np.min(vecx), np.max(vecx)
    stiple = np.linspace(start,stop)
    # returns a list of plots of the basis polynomials, overlaid
    return [plt.plot(stiple,[polys[i](t) for t in stiple])\
            for i in range(len(vecx))]

newton_basis_plot(vecx)

In practice we express the coordinate $a_n$ in recursive *divided differences*, i.e., $$a_n = f[x_0,\ldots,x_n] = \frac{f[x_1, \ldots, x_n] - f[x_0, \ldots, x_{n-1}]}{x_n-x_0}$$ where the identification $f[x] = f(x)$ for all $x$ is the base for recursion.

In [ ]:
def dd(f, vecx):
    if len(vecx)>1:
    # recursive call, returns the coordinate of the nth newton poly
        return (dd(f,vecx[1:]) - dd(f,vecx[:-1]))/(vecx[-1]-vecx[0])
    # floor of recursion
    else:
        return f(vecx[0])

def newton_coords(f, vecx):
    # TODO: save the recursive divided difference calls in memory
    # so we don't have to iterate to load them into this array
    return np.array([dd(f,vecx[:i+1]) for i in range(len(vecx))])

When working with more than a thousand generating points (i.e., for `len(vecx)` greater than `1000`), we'd have to rewrite the above programs to cache intermediate recursive function calls. The most straight forward means to cache the divided differences is in a triangular matrix, writing an imperative program to append new diagonals for the calculation of the $n$th coefficient $a_n$.

(A dynamically typed language, Python is not written to cache recursive calls, and generally taps out at 1000 levels deep. Pure functional, strictly typed languages like Haskell have compilers optimized to cache all recursions, and as well won't complain at any number of recursive calls.)

We note the Newton form, 
$$p_n(x) = a_0 + a_1(x-x_0) + \cdots + a_n(x-x_0)\cdots(x-x_{n-1})$$
can be optimized also to be evaluated in  $O(n)$ operations, namely, $n$ multiplications and $n$ additions, by factoring 
$$p_n(x) =  a_0 + (x-x_0)\bigg(a_1 + (x-x_1)\Big(a_2+\cdots + a_n(x-x_{n-1})\cdots\Big)\bigg).$$

Our code presently *does not* redistribute parentheses to save computation, instead, it's written to compute each basis polynomial explicitly (in however many multiplications), scale them all, then sum. Such operations correspond closing with the matrix interpretation of the constraint $p_n = f$ at $n+1$ points $x_i$.

### Matrix interpretation

For $n+1$ generating points $(x_i,f(x_i))$, there is a unique interpolating polynomial $p_n$ of degree $n$.

But $p_n$ is just point in the vector space $\mathcal{P}_n$, so we should try to express the constraint that $p_n(x_i) = f(x_i)$ for all $i \in \{0,1,\ldots,n\}$ as $n+1$ linear equations to determine the coordinates of $p_n$. 

The constraint as a matrix equation (in any basis) is
$$A \begin{pmatrix}a_0\\ \vdots\\ a_n\end{pmatrix} = \begin{pmatrix}f(x_0)\\ \vdots\\ f(x_n)\end{pmatrix}.$$

We are required to choose a basis for $\mathcal{P}_n$, and we've thus far seen two:

- the Lagrange polynomials $\{\ell_0, \ell_1, \ldots, \ell_n\}$
- the basis of the Newton form $\{1, (x-x_0), \ldots, (x-x_n)\}$

For the Lagrange polynomial basis, we have simply $A = I$. In the Newton basis, $A$ is a lower triangular matrix. 

A third basis, the standard basis $\{1,x,x^2,\ldots, x^n\}$, yields a power series approximation to $f$, but corresponds to the highly ill-conditioned *Vandermode* matrix.

### Error analysis

Let $f \in C^{n+1}[a,b]$, the $x_i$ be $n+1$ distinct points in $[a,b]$, and suppose $p_n$ is a polynomial of degree at most $n$ that interpolates $f$ at the $x_i$. Given $x \in [a,b]$, there exists a point $\xi \in (a,b)$ such that $$f(x) = p_n(x) + \frac{f^{(n+1)}(\xi)}{(n+1)!}\prod_{\forall i}(x-x_i).$$

For example, interpolating $\lambda x \mapsto e^x$ on $[0,3]$ with the generating points